Problem 1:CSV File encoding

In [ ]:
class NumericEmailEncrypter:
    def __init__(self):
        self.encryption_offset = 42
        # Valid ASCII range for email characters (43 to 122)
        self.valid_char_range = range(43, 123)

    def encrypt_email_id(self, email_id):
        """Encrypt the given email ID into a numeric string."""
        if not email_id or any(ord(char) not in self.valid_char_range for char in email_id):
            return "Invalid characters in email ID. Cannot encrypt."

        # Use list comprehension for faster encryption
        encrypted_email_id = "1" + "".join(
            f"{ord(char) - self.encryption_offset:02d}" for char in email_id
        )
        return encrypted_email_id

    def decrypt_email_id(self, encrypted_email_id):
        """Decrypt the numeric string back to the original email ID."""
        # Check if string starts with '1' and has valid length (even digits after '1')
        if not encrypted_email_id or encrypted_email_id[0] != '1' or len(encrypted_email_id) % 2 == 0:
            return "Invalid encrypted string. Cannot decrypt."

        # Remove the leading '1' and process in chunks of 2
        encrypted_email_id = encrypted_email_id[1:]
        try:
            # Faster decryption with list comprehension
            decrypted_email_id = "".join(
                chr(int(encrypted_email_id[i:i+2]) + self.encryption_offset)
                for i in range(0, len(encrypted_email_id), 2)
            )
            return decrypted_email_id
        except (ValueError, TypeError):
            return "Decryption failed. Invalid data format."


#  Example Usage
encrypter = NumericEmailEncrypter()

# Encrypt an email ID
email_id = "test.email@example.com"
encrypted_email = encrypter.encrypt_email_id(email_id)
print(f"Encrypted Email ID: {encrypted_email}")

# Decrypt the encrypted email ID
decrypted_email = encrypter.decrypt_email_id(encrypted_email)
print(f"Decrypted Email ID: {decrypted_email}")


**Encoder**

In [ ]:
from numeric_encrypter import NumericEmailEncrypter


def encode_file(input_file_name, output_file_name):
    """Encrypt email IDs in a CSV file and save the result."""
    try:
        # Initialize the encrypter
        encrypter = NumericEmailEncrypter()

        # Open input and output files using 'with' for automatic closure
        with open(input_file_name, 'r') as in_file, open(output_file_name, 'w') as out_file:
            # Write the header to the output file
            out_file.write("user_id,product_id,score\n")

            # Process each line after the header
            for line in in_file:
                values = line.strip().split(',')

                # Skip header row
                if values[0] == "user_email_id":
                    continue

                # Encrypt email and update the line
                encrypted_email = encrypter.encrypt_email_id(values[0])
                if encrypted_email is not None:
                    values[0] = encrypted_email
                    out_file.write(','.join(values) + "\n")
                else:
                    print(f"Skipping invalid email: {values[0]}")

        print(f"Encoding complete. Encrypted data saved to '{output_file_name}'.")

    except FileNotFoundError:
        print(f"❌ Error: File '{input_file_name}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


# Run the encoding function
encode_file("our_data.csv", "encoded_data.csv")


**Decoder**

In [ ]:
from numeric_encrypter import NumericEmailEncrypter


def decode_file(input_file_name, output_file_name):
    """Decrypt email IDs in a CSV file and save the result."""
    encrypter = NumericEmailEncrypter()

    try:
        # Open input and output files using 'with' for automatic closure
        with open(input_file_name, 'r') as in_file, open(output_file_name, 'w') as out_file:
            # Read and validate the header
            header = in_file.readline().strip()
            if header != "user_id,product_id,score":
                print("Invalid file format. Skipping decoding.")
                return

            # Write the correct header for the decoded file
            out_file.write("user_email_id,product_id,score\n")

            # Process each line after the header
            for line in in_file:
                values = line.strip().split(',')

                # Validate row format (3 columns expected)
                if len(values) != 3:
                    print(f"Skipping invalid row: {line.strip()}")
                    continue

                # Decrypt the email ID and check for validity
                decrypted_email = encrypter.decrypt_email_id(values[0])
                if decrypted_email:
                    values[0] = decrypted_email
                    out_file.write(','.join(values) + "\n")
                else:
                    print(f"Skipping invalid encrypted ID: {values[0]}")

        print(f"Decoding complete. Decrypted data saved to '{output_file_name}'.")

    except FileNotFoundError:
        print(f"❌ Error: File '{input_file_name}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


# Run the decoding function
decode_file("encoded_data.csv", "decoded_data.csv")


Sample input data csv file



```
user_email_id,product_id,score
a@gmail.com,1,4
b@gmail.com,2,3.5
c@gmail.com,3,2.0
```



Encoder file output

```
user_id,product_id,score
17375566958622273556770665904576967,1,4
1735568585959702257666975587866555604576967,2,3.5
17369675969685922616755636604576967,2,2.0
```



Decoder file output



```
user_email_id,product_id,score
a@gmail.com,1,4
b@gmail.com,2,3.5
c@gmail.com,3,2.0
```

